In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pf
from pathlib import Path
from hdbscan import HDBSCAN

sys.path.append(os.path.expanduser('~/Documents/Weave'))
from Weave.inputoutput import Reader
from Weave.clustering import Clustering, Latlons

In [ ]:
# current params: ms 2000, mcs 4000, allow True, epsilon = 0.0
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/t850_nhblock.7.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldseparate = ds['correlation'].sel(lag = -10, fold = 1)
fieldwave = ds['correlation'].sel(lag = -7, fold = 0)
clusterkwargs = dict(min_cluster_size=2000, #400 
                     min_samples=1000, # 1000 good noise reduction
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.17, #0.15# Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     core_dist_n_jobs = 7)

In [ ]:
fieldseparate.plot()

In [ ]:
fieldwave.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldseparate.expand_dims(dim = 'lag', axis = 0), mask = ~fieldseparate.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clusters = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clw = Clustering()
clw.reshape_and_drop_obs(array = fieldwave.expand_dims(dim = 'lag', axis = 0), mask = ~fieldwave.isnull(), min_samples = 700)
clw.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clustersw = clw.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
clusters.squeeze().plot()

In [ ]:
clustersw.squeeze().plot()

1000 ms gives a good first noise reduction.
At 2000 mcs the clustering for the sparse field seems ok (3 separate duo's, not together). However, the wave field is still subdivided (nothing is together)
At 2500, (i.c.w. a reasonable epsilon) sometimes a blob from too far away joins
at 3000 mcs the wave fields sees disappearing parts of the wave, still not together
at 4000 and 5000 still not together.
at 6000 another part disappears
0.15 epsilon works for connecting the wave
0.1 works only for three parts
0.175 works to connect part of another wave
Idea: 1000ms 2000mcs 0.17 epsilon (as the bare minimum to connect in these examples)

### Lets move to SST

In [ ]:
# current params: ms 2000, mcs 4000, allow True, epsilon = 0.0
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/sst_nhplus.7.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldmorenoise = ds['correlation'].sel(lag = -28, fold = 0)
fieldsignal = ds['correlation'].sel(lag = -12, fold = 1)
clusterkwargs = dict(min_cluster_size=1000, #300 worked good without epsilon
                     min_samples=300, # 250 good noise reduction without any epsilon, 300 with epsilon
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.22, #0.15# Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsignal.plot()

In [ ]:
fieldmorenoise.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsignal.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsignal.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clusters = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
cln = Clustering()
cln.reshape_and_drop_obs(array = fieldmorenoise.expand_dims(dim = 'lag', axis = 0), mask = ~fieldmorenoise.isnull(), min_samples = 700)
cln.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clustersn = cln.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
clusters.squeeze().plot()

In [ ]:
clustersn.squeeze().plot()

In [ ]:
fieldmorenoise.plot()

In [ ]:
fieldmorenoise.plot()

Sst will have a lower minimum sample size. As just the spatial coherence of anomalies is much less.
currently ms 300.
Did not make mcs too high. 
epsilon of 0.2 seems to work out well (6) clusters. But when the merge happens suddenly bit more noisy stuff included. Does ms help? -> well it removes some of the connecting elements. therefore some joined by epsilon split off again. This can again be counteracted with increasing mcs a bit.
working with very spotted fields: ms 400, mcs 500, epsilon 0.2
does not work so well for a less 'spotted' field 
therefore: slightly higher epsilon again: 0.22
and ms = 300, and mcs = 1000

### Onto a next challenging variable?

In [ ]:
# Variables with different resolution. e.g. snowcover. swvl
# Perhaps snowcover with 2-3 big regions and one with only a single.

In [ ]:
# current params: ms 2000, mcs 4000, allow True, epsilon = 0.0
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/snowc_nhmin.21.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldsingle = ds['correlation'].sel(lag = -21, fold = 0) # lag = -52, fold = 3
fieldmore = ds['correlation'].sel(lag = -52, fold = 0)
clusterkwargs = dict(min_cluster_size= 2500, # 1000 works already okay ish. I believe the single field is not truly single. Only starts to make a difference above 2500.
                     min_samples= 1000, # 1000 seems reasonable, (for mcs 200 and 1000) Fields are pretty connected with lots of spots.
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.1, ## Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsingle.plot()

In [ ]:
fieldmore.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsingle.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsingle.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clusterss = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clm = Clustering()
clm.reshape_and_drop_obs(array = fieldmore.expand_dims(dim = 'lag', axis = 0), mask = ~fieldmore.isnull(), min_samples = 700)
clm.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
clustersm = clm.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
clusterss.squeeze().plot()

In [ ]:
clusterss.squeeze().plot()

In [ ]:
clustersm.squeeze().plot()

In [ ]:
clustersm.squeeze().plot()

So the epsilon here is slightly less, because this variable specifically focuses also on the polar part of the northern hemisphere where we want to keep things apart (like siberia from america) and because fields are pretty dense. As expected ms and mcs can be a bit larger.
epsilon 0.1, ms = 1000, mcs = 1000 till 25000

### Siconc

In [ ]:
# current params: ms 2000, mcs 4000, allow True, epsilon = 0.0
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/siconc_nhmin.21.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldsignal = ds['correlation'].sel(lag = -52, fold = 0) # lag = -52, fold = 3
fieldless = ds['correlation'].sel(lag = -26, fold = 2)
clusterkwargs = dict(min_cluster_size= 400, # 300 is okay. But 400 removes some loose single cluster which is also desired (in the sense of less predictors)
                     min_samples= 200, # 150 seems reasonable, (for mcs 300) siconc Fields are pretty sparse.
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.1, ## Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsignal.plot()

In [ ]:
fieldless.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsignal.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsignal.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
signal = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clm = Clustering()
clm.reshape_and_drop_obs(array = fieldless.expand_dims(dim = 'lag', axis = 0), mask = ~fieldless.isnull(), min_samples = 700)
clm.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
less = clm.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
signal.squeeze().plot()

In [ ]:
less.squeeze().plot()

Siconc is tuned fairly not strict at the moment. Same epsilon as snowcover (0.1) which is nice I think as we are looking at the same region. Mcs = 400 and ms = 200, because of different resolutions of course, and because fields are not very dense. Could however be tuned stricter.

### Z300
at 15 simultaneous has a wave, other lags/folds have equator and scattered field

In [ ]:
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/z300_nhnorm.15.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldsignal = ds['correlation'].sel(lag = -15, fold = 3) # Interesting because part equator part wave, which we want to separate.
fieldless = ds['correlation'].sel(lag = -30, fold = 1)
clusterkwargs = dict(min_cluster_size= 5000, # 
                     min_samples= 2000, # 1200 works okay. But need a way to get rid of the isolated blobs. This is already better at 2000 (i.c.w. mcs 4000)
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.15, ## Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     cluster_selection_method='leaf',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsignal.plot()

In [ ]:
fieldless.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsignal.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsignal.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
signal = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clm = Clustering()
clm.reshape_and_drop_obs(array = fieldless.expand_dims(dim = 'lag', axis = 0), mask = ~fieldless.isnull(), min_samples = 700)
clm.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
less = clm.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
signal.squeeze().plot()

In [ ]:
less.squeeze().plot()

In [ ]:
fieldless.plot()

An epsilon of 0.17 connects the wave with the equator, which is probably not what we want. 0.15 doesn't 
z300 needs a very large mcs otherwise totally separate smooth 'blobs' will remain a cluster on their own, with little value probably. What is also an issue is that with larger mcs the combination of eom + epsilon is crashing. Fix is https://github.com/scikit-learn-contrib/hdbscan/issues/370 but not released yet. Therefore has to resort to cluster_selection_method = 'leaf' here. So currently at ms = 2000 and mcs 5000.

### Land surface variables (swvl, transp)

In [ ]:
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/swvl13_europe.11.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldsignal = ds['correlation'].sel(fold = 2, lag = -42) # At least two interesting regions (france vs east and scandinavia)
fieldless = ds['correlation'].sel(fold = 1, lag = -22) # Most should be filtered out
clusterkwargs = dict(min_cluster_size= 300, # 300 works okayish
                     min_samples= 200, # Does not need to be too high.
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.07, ## Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     cluster_selection_method='eom',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsignal.plot()

In [ ]:
fieldless.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsignal.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsignal.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
signal = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clm = Clustering()
clm.reshape_and_drop_obs(array = fieldless.expand_dims(dim = 'lag', axis = 0), mask = ~fieldless.isnull(), min_samples = 700)
clm.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
less = clm.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
signal.squeeze().plot()

In [ ]:
less.squeeze().plot()

swvl13 is a tricky one. Because of the spottyness, especially in northern africa, ms acts as a pretty heavy filter. Also mcs of 400 + ms of 200 already cancels everything there. (Plus crashes in combination with 'eom'). So If you want to retain those sparser regions it should be below. Also epsilon does not need to be very big to connect regions in europe that are very adjacent. Therefore epsilon = 0.07 and mcs = 300 with ms = 200
Transpiration is much more connected. Might need other parameters?

### TCC

In [ ]:
corrpath = Path('/nobackup_1/users/straaten/correlation_cv_spearmanpar_varalpha_strict/tcc_europe.11.corr.nc')

In [ ]:
ds = xr.open_dataset(corrpath, decode_times = False)

In [ ]:
fieldsignal = ds['correlation'].sel(lag = -11, fold = 1) # Dipole should become one. Some marginal stuff filtered out
fieldless = ds['correlation'].sel(lag = -32, fold = 4) # Perhaps lower right corner significant but other should be filtered out
clusterkwargs = dict(min_cluster_size=250, # 300 makes the less field completely empty works okayish
                     min_samples= 200, # Does not need to be too high.
                     allow_single_cluster=True,
                     cluster_selection_epsilon=0.09, ## Radian distance. Unit = fraction of earth radius. As 2pi * r is the full 2pi radian distance
                     metric='haversine',
                     cluster_selection_method='leaf',
                     core_dist_n_jobs = 7)

In [ ]:
fieldsignal.plot()

In [ ]:
fieldless.plot()

In [ ]:
cl = Clustering()
cl.reshape_and_drop_obs(array = fieldsignal.expand_dims(dim = 'lag', axis = 0), mask = ~fieldsignal.isnull(), min_samples = 700)
cl.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
signal = cl.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)
clm = Clustering()
clm.reshape_and_drop_obs(array = fieldless.expand_dims(dim = 'lag', axis = 0), mask = ~fieldless.isnull(), min_samples = 700)
clm.prepare_for_distance_algorithm(manipulator = Latlons, kwargs = {'to_radians':True})
less = clm.clustering(clusterclass = HDBSCAN, kwargs = clusterkwargs)

In [ ]:
signal.squeeze().plot()

In [ ]:
less.squeeze().plot()

Strange enough, again leaf seems needed sometimes. While mcs values are nowhere near the previous ones. I think that for the full run I do need to put selection algorithm at leaf.
Might be made a bit stricter the mcs. Although also the small blobs (which would disappear at mcs 300) seem consistent over the folds.
Epsilon = 0.09 (higher than swvl, but below e.g. snowc) ms = 200, mcs 250.